## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import gdown, zipfile, os, glob, shutil
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [ ]:
file_id = '1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl'
download_file(file_id, 'data.zip')
file_id = '1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl'
download_file(file_id, 'test.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl
From (redirected): https://drive.google.com/uc?id=1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl&confirm=t&uuid=8c03d7ca-5845-4cf0-9207-a9245e7f4ca1
To: /content/data.zip
100%|██████████| 289M/289M [00:02<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl
To: /content/test.zip
100%|██████████| 7.18M/7.18M [00:00<00:00, 63.7MB/s]


In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [ ]:
dataset_extract('data.zip')
dataset_extract('test.zip')

Extracting: 100%|██████████| 21/21 [00:00<00:00, 966.98files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
#test,train,valid 폴더 생성
import os
path = '/content/data/╗⌡ ╞·┤⌡/'

for folder in ['test','valid']:
    os.makedirs(path+folder+'/images',exist_ok=True)
    os.makedirs(path+folder+'/labels',exist_ok=True)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split

image_path = glob('/content/data/╗⌡ ╞·┤⌡/train/images/*')
len(image_path)

716

In [ ]:
train_img, valid_img, _, _ = train_test_split(image_path,image_path,test_size=0.2,random_state=2024)

len(train_img),len(valid_img)

(572, 144)

In [ ]:
train_img[1].split('/')[-1][:-3]+'txt'

'roof (1).txt'

In [ ]:
train_img[1].replace('cool_roof_images','cool_roof_yolo_labels').replace('jpg','txt')

'/content/data/╗⌡ ╞·┤⌡/train/images/roof (1).txt'

In [ ]:
# # train 파일 이동
# for img_path in train_img:
#     label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1][:-3]+'txt'
#     if os.path.exists(img_path):
#         shutil.move(img_path,path+'train/images')
#     if os.path.exists(label_path):
#         shutil.move(label_path,path+'train/labels')
# valid 파일 이동
for img_path in valid_img:
    label_path = '/content/data/╗⌡ ╞·┤⌡/train/labels/' + img_path.split('/')[-1][:-3]+'txt'
    if os.path.exists(img_path):
        shutil.move(img_path,path+'valid/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'valid/labels/')
# test 파일 이동
for img_path in glob('/content/test/test/*'):
    if os.path.exists(img_path):
        shutil.move(img_path,path+'test/images')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
import yaml

yaml_data = {'train' : '/content/data/╗⌡ ╞·┤⌡/train/images',
               'val' : '/content/data/╗⌡ ╞·┤⌡/valid/images',
               'names' : ['cool roof', 'generic roof'],
               'nc' : 2 }


yaml_file_path = "/content/data/╗⌡ ╞·┤⌡/data.yaml"
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import settings, YOLO

In [ ]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
model_transfer = YOLO(model='yolov8s.pt', task='detect')

In [ ]:
model_transfer.train(data='/content/data/╗⌡ ╞·┤⌡/data.yaml',
                    epochs=300,
                    patience=15,
                    seed=2024,
                    pretrained=True,
                    mixup = 0.3,
                    copy_paste = 0.3
                    )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/data/╗⌡ ╞·┤⌡/data.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/data/╗⌡ ╞·┤⌡/train/labels.cache... 572 images, 0 backgrounds, 1 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/data/╗⌡ ╞·┤⌡/train/images/KakaoTalk_20240412_100143804.mp4_20240412100854_0168.PNG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0194]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/╗⌡ ╞·┤⌡/valid/labels.cache... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train6
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     73/300      4.59G      1.209     0.9348      1.299        127        640: 100%|██████████| 36/36 [00:08<00:00,  4.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.97it/s]

                   all        144       1048      0.698      0.685      0.741      0.471



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     74/300      4.37G      1.234     0.9439      1.339        263        640:  67%|██████▋   | 24/36 [00:05<00:02,  4.11it/s]

In [ ]:
from ultralytics import YOLO
# from PIL import Image
# model_transfer = YOLO('/content/runs/detect/train3/weights/best.pt')

In [ ]:
image_path = '/content/data/╗⌡ ╞·┤⌡/test/images'
model_transfer.predict(source=image_path, save=True, iou=0.5)


image 1/21 /content/data/╗⌡ ╞·┤⌡/test/images/KakaoTalk_20240411_143843284.png: 640x576 5 cool roofs, 22 generic roofs, 98.3ms
image 2/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 112419.png: 576x640 4 generic roofs, 80.2ms
image 3/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 112543.png: 608x640 7 generic roofs, 79.8ms
image 4/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 112738.png: 640x640 1 cool roof, 3 generic roofs, 17.4ms
image 5/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 112832.png: 576x640 4 cool roofs, 2 generic roofs, 17.1ms
image 6/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 112921.png: 640x640 5 generic roofs, 17.4ms
image 7/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 113028.png: 608x640 6 cool roofs, 22 generic roofs, 17.4ms
image 8/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 2024-04-11 113048.png: 576x640 5 cool roofs, 5 generic roofs, 17.1ms
image 9/21 /content/data/╗⌡ ╞·┤⌡/test/images/╜║┼⌐╕░╝ª 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'cool roof', 1: 'generic roof'}
 obb: None
 orig_img: array([[[ 38,  31,  14],
         [ 38,  31,  14],
         [ 39,  32,  15],
         ...,
         [ 50,  53,  31],
         [ 57,  60,  38],
         [ 61,  64,  42]],
 
        [[ 39,  32,  15],
         [ 40,  33,  16],
         [ 41,  34,  17],
         ...,
         [ 54,  57,  35],
         [ 60,  63,  41],
         [ 64,  67,  45]],
 
        [[ 46,  39,  22],
         [ 46,  39,  22],
         [ 47,  40,  23],
         ...,
         [ 54,  57,  35],
         [ 60,  63,  41],
         [ 63,  66,  44]],
 
        ...,
 
        [[166, 169, 143],
         [164, 167, 141],
         [163, 167, 139],
         ...,
         [ 54,  47,  28],
         [ 48,  41,  22],
         [ 44,  37,  18]],
 
        [[165, 168, 142],
         [163, 166, 140],
         [163, 167, 138],
         .

In [ ]:
model_transfer.export(format='torchscript')

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 2.1s, saved as 'runs/detect/train/weights/best.torchscript' (42.9 MB)

Export complete (3.6s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=640 data=/content/data/╗⌡ ╞·┤⌡/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [ ]:
from PIL import Image